In [1]:
%load_ext autoreload
%autoreload 2

from _header import *

In [ ]:
"""
Load Huggingface dataset for streaming and reload download state
"""

dataset_mdcath = load_dataset("compsciencelab/mdCATH", split="train", streaming=True)
dataset_mdcath = dataset_mdcath.cast_column("image", Image(decode=False))

path_state_dict = "../tmp/data/3Di/_state.pkl"

if os.path.exists(path_state_dict):
    with open(path_state_dict, "rb") as f:
        state = pickle.load(f)
    print("State loaded,", state)
    dataset_mdcath = dataset_mdcath.skip(state)
else:
    state = 0
    print("No state found", state)

In [ ]:
"""
This script processes a dataset using multiprocessing to parallelize the workload.
"""

print("CPU Count: ", multiprocessing.cpu_count())

iterations = 1
urls = [item["image"]["path"] for item in dataset_mdcath.take(iterations)]

pool = multiprocessing.Pool(processes=multiprocessing.cpu_count() - 1 or 1)

dataset_mdcath_mapped = pool.map(
    # partial(download_process_pipeline, config=PROCESSING_CONFIG),
    download_process_pipeline,
    urls
)
pool.close()
pool.join()

In [ ]:
data = download(
    url='hf://datasets/compsciencelab/mdCATH@2393a6d9aebd04b688022d7a681b7c96d608de79/data/mdcath_dataset_1a2nA02.h5',
    # config=PROCESSING_CONFIG,
)

# ["image"]["path"]

In [2]:
process_pipeline()

<function src.data.mdcath_processing.process_pipeline(md_cath_hdf5: bytes)>

In [ ]:
state = state + iterations
with open(path_state_dict, "wb") as f:
    pickle.dump(state, f)
print(dataset_mdcath.state_dict())
print("State saved", state)